In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from io import StringIO
from IPython.display import display, clear_output

upload = None
output = widgets.Output()

def handle_upload(change):
    global upload
    try:
        file = next(iter(u.value))
        content = bytes(file['content']).decode('utf-8')
        upload = pd.read_csv(StringIO(content), sep='\t', skiprows=1)
        FRET(upload, d_t, a_t, bd, ba, cross)
        #FRET()
    except Exception as e:
        with output:
            output.clear_output(wait=True)
            print(f"Ошибка загрузки файла: {e}")

def FRET(*args):
    output.clear_output(wait=True)
    if upload is None:
        with output:
            print("Загрузите файл")
        return
    
    try:
        data = upload.copy()
     
        d, a = data.columns[3], data.columns[1]
        td, ta = data.columns[2], data.columns[0] #time columns

        dt = float(d_t.value)
        at = float(a_t.value)
        Ba = float(ba.value)
        Bd = float(bd.value)
        c = float(cross.value)
  
        data[ta] = pd.to_numeric(data[ta], errors='coerce')
        data[td] = pd.to_numeric(data[td], errors='coerce')
        data[a] = pd.to_numeric(data[a], errors='coerce')
        data[d] = pd.to_numeric(data[d], errors='coerce')
    
        data = data[(data[ta] == data[td])]

        data['Fa'] = data[a] - Ba
        data['Fd'] = data[d] - Bd - c * data[a]
        data['E'] = data['Fa']/(data['Fa']  + data['Fd'])
        valid = data[(data['Fa'] > at) & (data['Fd'] > dt)]
        
        with output:
            plt.hist(valid['E'], bins=100, alpha=0.7)
            plt.title('FRET Histogram')
            plt.xlabel('FRET Values')
            plt.ylabel('Frequency')
            plt.show()
    except Exception as e:
        with output:
            print(f"Error: {str(e)}")

back = widgets.HTML("<div style='text-align: left'>Background</div>", 
                   layout=widgets.Layout(width='auto', grid_area='back'))
tresh = widgets.HTML("<div style='text-align: left'>Treshhold</div>", 
                   layout=widgets.Layout(width='auto', grid_area='tresh'))
crosst = widgets.HTML('Crosstalk', 
                   layout=widgets.Layout(width='auto', grid_area='crosst'))
don = widgets.HTML("<div style='text-align: left'>Donor</div>", 
                   layout=widgets.Layout(width='auto', grid_area='don'))
acc = widgets.HTML("<div style='text-align: left'>Acceptor</div>", 
                   layout=widgets.Layout(width='auto', grid_area='acc'))

bd = widgets.Text(value='1.2',
                 layout=widgets.Layout(width='auto', grid_area='bd'))
ba = widgets.Text(value='0.9',
                  layout=widgets.Layout(width='auto', grid_area='ba'))
d_t = widgets.Text(value='0.9', 
                   layout=widgets.Layout(width='auto', grid_area='d_t'))
a_t = widgets.Text(value='0.9',
                   layout=widgets.Layout(width='auto', grid_area='a_t'))
cross = widgets.Text(value='0.21', 
                     layout=widgets.Layout(width='auto', grid_area='cross'))
u = widgets.FileUpload(accept='txt',multiple=False,
                             layout=widgets.Layout(width='auto', grid_area='u'))

grid = widgets.GridBox(
    children=[back, tresh, don, acc, crosst, bd, ba, d_t, a_t, cross, u],
    layout=widgets.Layout(
        width='100%',
        grid_template_columns='20% 10% 20% 20% 20%',
        grid_template_rows='auto auto auto',       # 2 rows
        grid_template_areas='''
            ". . back tresh crosst"
            "u don bd d_t cross"            
            ". acc ba a_t ."
        ''',
    )
)

u.observe(handle_upload, 'value')
for widget in [bd, ba, d_t, a_t, cross]:
    widget.observe(FRET, 'value')

ui = widgets.VBox([grid, output])
display(ui)
